## Import needed libraries

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from google.colab import drive
from google.colab import files

import os
import zipfile

## Read data

The data was uploaded to my drive.



In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define our example directories and files
base_dir = '/content/drive/MyDrive/tensorflow-course-2/My notebook/instruments'

In [ ]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 45,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   vertical_flip = True)

In [ ]:
pic_shape = 300

In [ ]:
# Download the pre-trained weights. No top means it excludes the fully connected layer it uses for classification.
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2023-09-07 12:57:04--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 142.250.141.207, 2607:f8b0:4023:c0d::cf, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   147MB/s    in 0.6s    

2023-09-07 12:57:05 (147 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

# Set the weights file you downloaded into a variable
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Initialize the base model.
# Set the input shape and remove the dense layers.
pre_trained_model = InceptionV3(input_shape = (300, 300, 3),
                                include_top = False,
                                weights = None)

# Load the pre-trained weights you downloaded.
pre_trained_model.load_weights(local_weights_file)

# Freeze the weights of the layers.
for layer in pre_trained_model.layers:
  layer.trainable = False

In [ ]:
# Choose `mixed7` as the last layer of your base model
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 17, 17, 768)


In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate
x = layers.Dropout(0.4)(x)
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)

# Append the dense network to the base model
model = Model(pre_trained_model.input, x)

# Print the model summary. See your dense network connected at the end.
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
# Set the training parameters
model.compile(optimizer = RMSprop(learning_rate=0.0001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
 # All images will be rescaled by 1./255
# train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/tensorflow-course-2/My notebook/instruments/train',
        target_size=(pic_shape, pic_shape),  # All images will be resized to 300x300
        batch_size=128,
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        '/content/drive/MyDrive/tensorflow-course-2/My notebook/instruments/val',
        target_size=(pic_shape, pic_shape),  # All images will be resized to 300x300
        batch_size=32,
        class_mode='binary')

Found 2675 images belonging to 2 classes.
Found 307 images belonging to 2 classes.


In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=16,
      epochs=25,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

Epoch 1/25
16/16 [==============================] - 530s 32s/step - loss: 1.8956 - accuracy: 0.6157 - val_loss: 0.5976 - val_accuracy: 0.6641
Epoch 2/25
16/16 [==============================] - 114s 7s/step - loss: 0.5341 - accuracy: 0.7371 - val_loss: 0.4643 - val_accuracy: 0.7656
Epoch 3/25
16/16 [==============================] - 69s 4s/step - loss: 0.4393 - accuracy: 0.8000 - val_loss: 0.2605 - val_accuracy: 0.9336
Epoch 4/25
16/16 [==============================] - 60s 4s/step - loss: 0.3855 - accuracy: 0.8280 - val_loss: 0.2387 - val_accuracy: 0.9102
Epoch 5/25
16/16 [==============================] - 57s 3s/step - loss: 0.3550 - accuracy: 0.8531 - val_loss: 0.2632 - val_accuracy: 0.8984
Epoch 6/25
16/16 [==============================] - 61s 4s/step - loss: 0.3266 - accuracy: 0.8595 - val_loss: 0.1934 - val_accuracy: 0.9375
Epoch 7/25
16/16 [==============================] - 61s 4s/step - loss: 0.2993 - accuracy: 0.8791 - val_loss: 0.1734 - val_accuracy: 0.9258
Epoch 8/25
16/16 

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = fn
  img = load_img(path, target_size=(300, 300))
  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes)

Saving guitar (1).jpg to guitar (1) (1).jpg
Saving guitar (2).jpg to guitar (2) (1).jpg
Saving guitar (3).jpg to guitar (3) (1).jpg
Saving guitar (4).jpg to guitar (4) (1).jpg
Saving guitar (5).jpg to guitar (5) (1).jpg
Saving guitar (6).jpg to guitar (6) (1).jpg
Saving guitar (7).jpg to guitar (7) (1).jpg
Saving guitar (8).jpg to guitar (8) (1).jpg
Saving guitar (9).jpg to guitar (9) (1).jpg
Saving guitar (10).jpg to guitar (10) (1).jpg
Saving guitar (11).jpg to guitar (11) (1).jpg
Saving guitar (12).jpg to guitar (12) (1).jpg
Saving guitar (13).jpg to guitar (13) (1).jpg
Saving guitar (14).jpg to guitar (14) (1).jpg
Saving guitar (15).jpg to guitar (15) (1).jpg
Saving guitar (16).jpg to guitar (16) (1).jpg
Saving guitar (17).jpg to guitar (17) (1).jpg
Saving guitar (18).jpg to guitar (18) (1).jpg
Saving guitar (19).jpg to guitar (19) (1).jpg
Saving guitar (20).jpg to guitar (20) (1).jpg
Saving guitar (21).jpg to guitar (21) (1).jpg
Saving guitar (22).jpg to guitar (22) (1).jpg
Saving